In past missions, we focused on exploring the SQL syntax for retrieving data from a database. In this mission, we'll explore **how to interact with a SQLite database in Python** so you can start to incorporate databases into your data science workflow.
<br>

**SQLite is a database that doesn't require a standalone server**; it stores the entire database as a file on disk. This makes it **ideal for working with larger data sets that can fit on disk but not in memory.**<br>

The pandas library loads the entire data set we're working with into memory, **making SQLite a compelling alternative for working with data sets larger than 8 gigabytes** (which is roughly the amount of memory modern computers contain). The fact that we can contain an entire database in a single file makes them easy to share; **some data sets are available online as SQLite database files** (using the extension `.db`).
<br>

We can interact with a SQLite database in two main ways:

* Through the [sqlite3 Python module](https://docs.python.org/3/library/sqlite3.html)
* Through the [SQLite shell](https://sqlite.org/cli.html)

In this mission, we'll focus on learning how to use the sqlite3 module to interact with the database.

We've loaded a subset of the data into a table named `recent_grads` in a database. The subset contains the 2010-2012 data for recent college grads only. 
### The database file we'll be working with is called jobs.db.

Python 2.5 and up come with the sqlite module, which means we don't need to install any separate libraries to get started. Specifically, we'll be working with the sqlite3 Python module, which was developed to work with [SQLite version 3](https://www.sqlite.org/version3.html).

We can import it into our environment using this command:

In [1]:
import sqlite3

Once we've imported the module, we connect to the database we want to query using the `connect()` function. This function requires a single parameter, which is the database we want to connect to. Because the database we're working with exists as a file on disk, we need to pass in the file name
<br>

The `connect()` function returns a [Connection instance](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection), which maintains the connection to the database we want to work with. When we're connected to a database, SQLite locks the database file and prevents any other processes from connecting to the database simultaneously. The SQLite team made this design decision to keep the database lightweight, and avoid the complexity that arises when multiple processes interact with the same database.

#### instructions
* Import the sqlite3 library into the environment.
* Then, use the `sqlite3.connect()` function to connect to `jobs.db`, and assign the Connection instance it returns to `conn`.

```sqlite3
import sqlite3
conn = sqlite3.connect("jobs.db")
```

Before we can execute a query, we need to express our SQL query as a string. While we use the Connection class to represent the database we're working with, we use the [Cursor class](https://docs.python.org/3/library/sqlite3.html#cursor-objects) to:

* Run a query against the database
* Parse the results from the database
* Convert the results to native Python objects
* Store the results within the Cursor instance as a local variable

After running a query and converting the results to **a list of tuples, the Cursor instance stores the list as a local variable**. Before diving into the syntax of querying the database, let's learn more about tuples.

A [tuple](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences) is a core data structure that Python uses to represent a sequence of values, similar to a list. Unlike lists, tuples are **immutable**, which means we can't modify existing ones. Python represents each row in the results set as a tuple.

To create an empty tuple, assign a pair of empty parentheses to a variable:
```python
t = ()
```

Python indexes Tuples from 0 to n-1, just like it does with lists. We access the values in a tuple using bracket notation.

```python
t = ('Apple', 'Banana')
apple = t[0]
banana = t[1]
```

### Tuples are faster than lists
* so they'are helpful with larger databases and larger results sets.



We need to use the Connection instance method `cursor()` to return a Cursor instance corresponding to the database we want to query.

```python
cursor = conn.cursor()
```

In the following code block, we:

* Write a basic `select` query that will return all of the values from the `recent_grads` table, and store this query as a string named `query`
* Use the Cursor method `execute()` to run the query against our database
* Return the full results set and store it as `results`
* Print the first three tuples in the list `results`

```python
# SQL Query as a string
query = "select * from recent_grads;"

# Execute the query, convert the results to tuples, and store as a local variable
cursor.execute(query)

# Fetch the full results set as a list of tuples
results = cursor.fetchall()

# Display the first three results
print(results[0:3])
```

#### instructions
* Write a query that returns all of the values in the Major column from the recent_grads table.
* Store the full results set (a list of tuples) in majors.
* Then, print the first three tuples in majors.


```python
import sqlite3
conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()

query = "select * from recent_grads;"
cursor.execute(query)
results = cursor.fetchall()
print(results[0:2])

query = "select Major from recent_grads;"
cursor.execute(query)
majors = cursor.fetchall()
print(majors[:3])

'''output
[('PETROLEUM ENGINEERING',), ('MINING AND MINERAL ENGINEERING',), ('METALLURGICAL ENGINEERING',)]
'''
```

So far, we've been running queries by creating a Cursor instance, and then calling the `execute` method on the instance. The SQLite library actually allows us to skip creating a Cursor altogether by using the [`execute` method](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.execute) within the Connection object itself. SQLite will create a Cursor instance for us under the hood and our query run against the database, but this shortcut allows us to skip a step. Here's what the code looks like:

```python
conn = sqlite3.connect("jobs.db")
query = "select * from recent_grads;"
conn.execute(query).fetchall()
```
Notice that we didn't explicitly create a separate Cursor instance ourselves in this code example.
<br>

Now let's learn how to fetch a specific number of results after we run a query.

### To make it easier to work with large results sets, 
the Cursor class allows us to control the number of results we want to retrieve at any given time. To return a single result (as a tuple), we use the [Cursor method fetchone()](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchone). To return `n` results, we use the [Cursor method fetchmany()](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchmany).
<br>

Each Cursor instance contains an internal counter that updates every time we retrieve results. When we call the `fetchone()` method, the Cursor instance will return a single result, and then increment its internal counter by 1. This means that if we call `fetchone()` again, the Cursor instance will actually return the second tuple in the results set (and increment by 1 again).
<br>

The `fetchmany()` method takes in an integer (`n`) and returns the corresponding results, starting from the current position. It then increments the Cursor instance's counter by `n`. In the following code, we return the first two results using the `fetchone()` method, then the next five results using the `fetchmany()` method.

```python
first_result = cursor.fetchone()
second_result = cursor.fetchone()
next_five_results = cursor.fetchmany(5)
```

* Write and run a query that returns the Major and Major_category columns from recent_grads.
* Then, fetch the first five results and store them as five_results.

```python
import sqlite3
conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()

five_results = cursor.execute("""
        select Major, Major_category
        from recent_grads""").fetchmany(5)
```

* Close the connection to the database using the Connection instance method close().

```python
conn.close()
```

* Connect to the database jobs2.db, which contains the same data as jobs.db.
* Write and execute a query that returns all of the majors (Major) in reverse alphabetical order (Z to A).
* Assign the full result set to reverse_alphabetical.

Finally, close the connection to the database.

In [ ]:
import sqlite3

conn = sqlite3.connect("jobs2.db")

query = """
        select Major from recent_grads order by Major desc;
"""

reverse_alphabetical = conn.cursor().execute(query).fetchall()

conn.close()